# Import relevant libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# xgboost
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# PCA
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import PCA
from kneed import KneeLocator

# logreg / rfe
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# to display all rows in dataframes
pd.set_option('display.max_rows', None) 

SyntaxError: invalid syntax (xgboost.py, line 43)

# Load data

In [ ]:
df = pd.read_csv("../data/grouped_data.csv")
X_train = pd.read_csv("../data/X_train_final.csv")
X_test = pd.read_csv("../data/X_test_final.csv")
y_train = pd.read_csv("../data/y_train_final.csv")
y_test = pd.read_csv("../data/y_test_final.csv")

In [ ]:
# one hot encode the categories
features_nominal = ['order_1', 'order_2', 'order_3', 'order_6', 'order_7']
X_train = pd.get_dummies(X_train, columns = features_nominal)
X_test = pd.get_dummies(X_test, columns = features_nominal)

# Feature Selection

In [ ]:
impt_feat = []

## XGB feature importance

In [ ]:
# fit model to training data
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X_train.columns, xgb.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame(feats.items(), columns=['Feature', 'Importance'])
#.rename(columns={0: 'importance'})
importances = importances.sort_values(by = ['Importance'], ascending = False)
impt_feat.extend(importances.Feature.iloc[0:30].tolist())

In [ ]:
impt_feat

## PCA dimensionality reduction

Remove categorical features

In [ ]:
df_pca = df.drop(columns = ['label', 'sevenmers', 'gene_id', 'transcript_id', 'order_1', 'order_2', 'order_3', 'order_6', 'order_7'])

In [ ]:
_ , p_value = calculate_bartlett_sphericity(df_pca)
p_value

In [ ]:
_, kmo_score = calculate_kmo(df_pca)
kmo_score 

As p-value < 0.5 and kmo score > 0.5, PCA is suitable on the dataframe.

In [ ]:
fa = FactorAnalyzer(n_factors = 10, method = 'principal', rotation='varimax')
fa.fit(df_pca)
eigenvalues, _ = fa.get_eigenvalues()
variances = fa.get_factor_variance()

In [ ]:
x = list(range(1,11))
plt.figure(figsize=(10, 7)) 
plt.bar(x,variances[2])
plt.title('Cumulative Variance')
plt.xlabel('PC Number')
plt.ylabel('Proportion of Variance Explained by PC')

In [ ]:
def evaluate_pcs(num_of_pcs,data):
    def encode_vals(x):
        if x <= -0.7 or x >= 0.7:
            return x
        else:
            return("")    
    # REMARK: we use 'principal' method and 'varimax' rotation in the FactorAnalyzer function.
    f = FactorAnalyzer(n_factors=num_of_pcs, method = 'principal',rotation='varimax')
    f.fit(data)
    loadings = pd.DataFrame(f.loadings_).set_index(data.columns)
    loadings = loadings.applymap(encode_vals)
    loadingcols= list(loadings.columns)
    newcols = {}
    for i in loadingcols:
        newcols[i] = "PC" + str(i+1)
    loadings.rename(columns = newcols,inplace=True)
    return loadings

# The following function generates the rotation matrix. Recall that we use
# this matrix to determine if the PCs generated are easily understandable and appropriate.
# The argument "num_of_pcs" specifies, the number of PCs we wish to generate.

In [ ]:
def variance_explained(num_of_pcs,data):
    # REMARK: we use 'principal' method and 'varimax' rotation in the FactorAnalyzer function.
    f = FactorAnalyzer(n_factors=num_of_pcs, method = 'principal',rotation='varimax')
    f.fit(data)
    return f.get_factor_variance()[2][num_of_pcs-1]

# The following function calculates the variance explained by the specified desired number of PCs.

Chose 9 PCs as it explains >70% of the variance

In [ ]:
variance_explained(9, df_pca)

In [ ]:
evaluate_pcs(9,df_pca)

Drop the columns that are not in any PCs as they are not as important.

In [ ]:
new_df = df.drop(columns = ['transcript_position', 'dwelling_time_1_min',
                           'sd_current_1_min', 'sd_current_1_max', 'sd_current_1_std',
                           'mean_current_1_std', 'dwelling_time_2_min', 'dwelling_time_2_median',
                           'sd_current_2_min', 'mean_current_2_min', 'mean_current_2_max', 'mean_current_2_median',
                           'mean_current_2_std', 'dwelling_time_3_min', 'sd_current_3_min',
                           'mean_current_3_std', 'diff_dwelling_time_1_median', 'diff_dwelling_time_1_std',
                           'diff_dwelling_time_2_median', 'diff_sd_current_1_min', 'diff_sd_current_1_median',
                           'diff_mean_current_1_max', 'diff_mean_current_1_std', 'diff_mean_current_2_min',
                           'diff_mean_current_2_median', 'relative_position', 'count_A', 'count_C', 'count_G', 'count_T'])

In [ ]:
cols2keep_pca = new_df.columns
impt_feat.extend(new_df.columns)

## RFE Recursive Feature Elimination

## 

In [ ]:
logreg = LogisticRegression(max_iter=1000)

In [ ]:
rfe3 = RFE(logreg, n_features_to_select=30)
rfe3 = rfe3.fit(X_train, y_train.values.ravel())

## 

In [ ]:
# cols remaining
cols_keep = X_train.columns.values[rfe3.support_]
impt_feat.extend(cols_keep)

In [ ]:
impt_feat

In [ ]:
len(impt_feat)

## Feature Importance using Random Forest

In [ ]:
# baseline model with default parameters
start = time.time()
forest1 = RandomForestClassifier(random_state = 1, n_jobs= -1)
forest1.fit(X_train,y_train.values.ravel())
end = time.time()
print("Time taken:", (end-start)/60, "minutes")

In [ ]:
rf_y_pred = forest1.predict(X_test)

In [ ]:
print(metrics.confusion_matrix(y_test, rf_y_pred))
# TN FP
# FN TP

print(f'accuracy: {metrics.accuracy_score(y_test, rf_y_pred)}')
print(f'precision: {metrics.precision_score(y_test,rf_y_pred)}')
print(f'recall:    {metrics.recall_score(y_test, rf_y_pred)}')
print(f'roc auc:   {metrics.roc_auc_score(y_test, rf_y_pred)}')
print(f'pr auc:    {metrics.average_precision_score(y_test, rf_y_pred)}')

y_predict_prob = forest1.predict_proba(X_test)[:, 1]
print(metrics.confusion_matrix(y_test, rf_y_pred))
print(f'roc auc:   {metrics.roc_auc_score(y_test, y_predict_prob)}')
print(f'pr auc:    {metrics.average_precision_score(y_test, y_predict_prob)}')

In [ ]:
importance = forest1.feature_importances_
importance

In [ ]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X_train.columns, forest1.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame(feats.items(), columns=['Feature', 'Importance'])
#.rename(columns={0: 'importance'})
importances = importances.sort_values(by = ['Importance'], ascending = False)
importances

In [ ]:
importances.Feature.iloc[0: 40].tolist()